<a href="https://colab.research.google.com/github/MahimnaDarji/Cricket-Analysis/blob/main/Score%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
import numpy as np
import pandas as pd
import glob

import warnings
warnings.filterwarnings('ignore')

In [106]:
data=pd.read_csv('/content/drive/MyDrive/DATASETS/Data.csv')

In [107]:
data.dtypes

match_id                    int64
season                     object
start_date                 object
venue                      object
innings                     int64
ball                      float64
batting_team               object
bowling_team               object
striker                    object
non_striker                object
bowler                     object
runs_off_bat                int64
extras                      int64
wides                     float64
noballs                   float64
byes                      float64
legbyes                   float64
penalty                   float64
wicket_type                object
player_dismissed           object
other_wicket_type         float64
other_player_dismissed    float64
dtype: object

In [108]:
data.batting_team.unique()

array(['Ireland', 'West Indies', 'Pakistan', 'United Arab Emirates',
       'Papua New Guinea', 'Australia', 'New Zealand', 'India',
       'South Africa', 'England', 'Netherlands', 'Oman', 'Sri Lanka',
       'Bangladesh', 'Zimbabwe', 'Malta', 'Gibraltar', 'Kenya',
       'Malaysia', 'Canada', 'Nepal', 'Afghanistan', 'Scotland',
       'Austria', 'Luxembourg', 'Hong Kong', 'Bermuda', 'Namibia',
       'Uganda', 'Vanuatu', 'Jersey', 'ICC World XI', 'Philippines',
       'Guernsey', 'Denmark', 'Czech Republic', 'Germany', 'Bulgaria',
       'Romania', 'Nigeria', 'Belgium', 'Ghana', 'Rwanda', 'Thailand',
       'Cayman Islands', 'Botswana', 'Singapore', 'Estonia', 'Cyprus',
       'Italy', 'Malawi', 'Lesotho', 'Swaziland', 'Seychelles',
       'Sierra Leone', 'Switzerland', 'Qatar', 'Bahrain', 'Kuwait',
       'Maldives', 'Tanzania', 'Mozambique', 'Cameroon', 'Belize',
       'United States of America', 'Bahamas', 'Argentina', 'Hungary',
       'Sweden', 'Finland', 'France', 'Isle of Man

In [109]:
topteams=['Australia','West Indies','Pakistan','New Zealand','India','South Africa','England','Sri Lanka','Bangladesh']

In [110]:
data=data[data.batting_team.isin(topteams)]
data=data[data.bowling_team.isin(topteams)]

In [111]:
data=data.sort_values(['match_id','innings'],ascending=[True,True])

In [112]:
data['overs']=data['ball']
data['ball']=data['ball'].astype(str)
data['ball'].str[0:-1]

407406     0.
407407     0.
407408     0.
407409     0.
407410     0.
         ... 
222022    19.
222023    19.
222024    19.
222025    19.
222026    19.
Name: ball, Length: 165092, dtype: object

In [113]:
data['over']=data['ball'].str[0:-1]
data['over']=data['over'].str.replace('.','')
data['ball']=data['ball'].str[-1:]

In [114]:
data['total_runs']=data['extras']+data['runs_off_bat']

In [115]:
data=data[['match_id', 'innings', 'batting_team', 'bowling_team', 'overs', 'over', 'ball','extras','total_runs', 'player_dismissed']]

In [116]:
data=data.rename(columns={"match_id": "id"})

In [117]:
data = data.replace(np.nan, 0)

In [118]:
data['total']=data.groupby(['id', 'innings'])['total_runs'].transform('sum')

In [119]:
data['total_score']=data.groupby(['id', 'innings'])['total_runs'].apply(lambda x: x.cumsum())

In [120]:
data.head(5)

,id,innings,batting_team,bowling_team,overs,over,ball,extras,total_runs,player_dismissed,total,total_score
407406,211028,1,England,Australia,0.1,0,1,0,0,0,179,0
407407,211028,1,England,Australia,0.2,0,2,0,1,0,179,1
407408,211028,1,England,Australia,0.3,0,3,0,0,0,179,1
407409,211028,1,England,Australia,0.4,0,4,0,0,0,179,1
407410,211028,1,England,Australia,0.5,0,5,0,0,0,179,1


In [121]:
tmp=data.groupby(['id','innings'])['total_runs'].rolling(min_periods=1, window=30).sum().reset_index()

In [122]:
tmp[['total_runs']]

,total_runs
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
165087,37.0
165088,37.0
165089,34.0
165090,34.0


In [123]:
data['prev_30ball_runs']=tmp['total_runs'].to_list()

In [124]:
data['player_dismissed']=np.where(data['player_dismissed']==0, 0, 1)

In [125]:
data['total_wickets']=data.groupby(['id','innings'])['player_dismissed'].apply(lambda x: x.cumsum())

In [126]:
tmp=data.groupby(['id','innings'])['player_dismissed'].rolling(min_periods=1,window=30).sum().reset_index()

In [127]:
data['prev_30ball_wkts']=tmp['player_dismissed'].to_list()

In [128]:
data['dot_balls']=data['total_runs']
data['dot_balls']=np.where(data['dot_balls']==0,1,0)
tmp=data.groupby(['id','innings'])['dot_balls'].rolling(min_periods=1,window=30).sum().reset_index()
data['prev_30ball_dots']=tmp['dot_balls'].to_list()

In [129]:
data.head(20)

,id,innings,batting_team,bowling_team,overs,over,ball,extras,total_runs,player_dismissed,total,total_score,prev_30ball_runs,total_wickets,prev_30ball_wkts,dot_balls,prev_30ball_dots
407406,211028,1,England,Australia,0.1,0,1,0,0,0,179,0,0.0,0,0.0,1,1.0
407407,211028,1,England,Australia,0.2,0,2,0,1,0,179,1,1.0,0,0.0,0,1.0
407408,211028,1,England,Australia,0.3,0,3,0,0,0,179,1,1.0,0,0.0,1,2.0
407409,211028,1,England,Australia,0.4,0,4,0,0,0,179,1,1.0,0,0.0,1,3.0
407410,211028,1,England,Australia,0.5,0,5,0,0,0,179,1,1.0,0,0.0,1,4.0
407411,211028,1,England,Australia,0.6,0,6,1,1,0,179,2,2.0,0,0.0,0,4.0
407412,211028,1,England,Australia,0.7,0,7,0,2,0,179,4,4.0,0,0.0,0,4.0
407413,211028,1,England,Australia,1.1,1,1,0,0,0,179,4,4.0,0,0.0,1,5.0
407414,211028,1,England,Australia,1.2,1,2,0,0,0,179,4,4.0,0,0.0,1,6.0
407415,211028,1,England,Australia,1.3,1,3,1,1,0,179,5,5.0,0,0.0,0,6.0


In [130]:
data['boundaries']=data['total_runs']
data['boundaries']=np.where(data['boundaries']>3,1,0)
tmp=data.groupby(['id','innings'])['boundaries'].rolling(min_periods=1,window=30).sum().reset_index()
data['prev_30ball_bound']=tmp['boundaries'].to_list()

In [131]:
data.head(10)

,id,innings,batting_team,bowling_team,overs,over,ball,extras,total_runs,player_dismissed,total,total_score,prev_30ball_runs,total_wickets,prev_30ball_wkts,dot_balls,prev_30ball_dots,boundaries,prev_30ball_bound
407406,211028,1,England,Australia,0.1,0,1,0,0,0,179,0,0.0,0,0.0,1,1.0,0,0.0
407407,211028,1,England,Australia,0.2,0,2,0,1,0,179,1,1.0,0,0.0,0,1.0,0,0.0
407408,211028,1,England,Australia,0.3,0,3,0,0,0,179,1,1.0,0,0.0,1,2.0,0,0.0
407409,211028,1,England,Australia,0.4,0,4,0,0,0,179,1,1.0,0,0.0,1,3.0,0,0.0
407410,211028,1,England,Australia,0.5,0,5,0,0,0,179,1,1.0,0,0.0,1,4.0,0,0.0
407411,211028,1,England,Australia,0.6,0,6,1,1,0,179,2,2.0,0,0.0,0,4.0,0,0.0
407412,211028,1,England,Australia,0.7,0,7,0,2,0,179,4,4.0,0,0.0,0,4.0,0,0.0
407413,211028,1,England,Australia,1.1,1,1,0,0,0,179,4,4.0,0,0.0,1,5.0,0,0.0
407414,211028,1,England,Australia,1.2,1,2,0,0,0,179,4,4.0,0,0.0,1,6.0,0,0.0
407415,211028,1,England,Australia,1.3,1,3,1,1,0,179,5,5.0,0,0.0,0,6.0,0,0.0


In [132]:
tmp=data.groupby(['id','innings'])['extras'].rolling(min_periods=1,window=30).sum().reset_index()
data['prev_30ball_extras']=tmp['extras'].to_list()

In [133]:
data.head(10)

,id,innings,batting_team,bowling_team,overs,over,ball,extras,total_runs,player_dismissed,total,total_score,prev_30ball_runs,total_wickets,prev_30ball_wkts,dot_balls,prev_30ball_dots,boundaries,prev_30ball_bound,prev_30ball_extras
407406,211028,1,England,Australia,0.1,0,1,0,0,0,179,0,0.0,0,0.0,1,1.0,0,0.0,0.0
407407,211028,1,England,Australia,0.2,0,2,0,1,0,179,1,1.0,0,0.0,0,1.0,0,0.0,0.0
407408,211028,1,England,Australia,0.3,0,3,0,0,0,179,1,1.0,0,0.0,1,2.0,0,0.0,0.0
407409,211028,1,England,Australia,0.4,0,4,0,0,0,179,1,1.0,0,0.0,1,3.0,0,0.0,0.0
407410,211028,1,England,Australia,0.5,0,5,0,0,0,179,1,1.0,0,0.0,1,4.0,0,0.0,0.0
407411,211028,1,England,Australia,0.6,0,6,1,1,0,179,2,2.0,0,0.0,0,4.0,0,0.0,1.0
407412,211028,1,England,Australia,0.7,0,7,0,2,0,179,4,4.0,0,0.0,0,4.0,0,0.0,1.0
407413,211028,1,England,Australia,1.1,1,1,0,0,0,179,4,4.0,0,0.0,1,5.0,0,0.0,1.0
407414,211028,1,England,Australia,1.2,1,2,0,0,0,179,4,4.0,0,0.0,1,6.0,0,0.0,1.0
407415,211028,1,England,Australia,1.3,1,3,1,1,0,179,5,5.0,0,0.0,0,6.0,0,0.0,2.0


In [134]:
convert_dict={
    'prev_30ball_runs':'int',
    'prev_30ball_bound':'int',
    'prev_30ball_dots':'int',
    'prev_30ball_extras':'int',
    'prev_30ball_wkts':'int',
}
data=data.astype(convert_dict)

In [135]:
data.columns

Index(['id', 'innings', 'batting_team', 'bowling_team', 'overs', 'over',
       'ball', 'extras', 'total_runs', 'player_dismissed', 'total',
       'total_score', 'prev_30ball_runs', 'total_wickets', 'prev_30ball_wkts',
       'dot_balls', 'prev_30ball_dots', 'boundaries', 'prev_30ball_bound',
       'prev_30ball_extras'],
      dtype='object')

In [147]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [148]:
from sklearn.preprocessing import LabelEncoder
lbn=LabelEncoder()
X=data.iloc[:,2].values
data.iloc[:,2]=lbn.fit_transform(X)
X=X.reshape(-1,1)

from sklearn.preprocessing import LabelEncoder
lbn=LabelEncoder()
X=data.iloc[:,3].values
data.iloc[:,3]=lbn.fit_transform(X)
X=X.reshape(-1,1)

In [149]:
X=data[['overs','total_score','total_wickets','prev_30ball_bound','prev_30ball_dots','prev_30ball_extras','prev_30ball_runs','prev_30ball_wkts']]
Y=data['total'].values

In [150]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25)

In [151]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(123819, 8) (41273, 8) (123819,) (41273,)


In [152]:
LR_model = LinearRegression()
LR_model.fit(X_train, Y_train)

LinearRegression()

In [153]:
prediction=LR_model.predict(X_test)
mean_absolute_error (Y_test, prediction)

18.749776474244907

In [154]:
def score_prediction(overs,total_score,total_wickets,prev_30ball_bound,prev_30ball_dots,prev_30ball_extras,prev_30ball_runs,prev_30ball_wkts):
     
    temp_array = list()      
    temp_array = temp_array + [overs,total_score,total_wickets,prev_30ball_bound,prev_30ball_dots,prev_30ball_extras,prev_30ball_runs,prev_30ball_wkts]
    data1 = np.array([temp_array])
    my_prediction = int(LR_model.predict(data1))
    
    print('Average Predicted Score: ',my_prediction)
    print('Maximumm Predicted Score:',my_prediction + 18)

In [155]:
overs= 7
total_score= 72
total_wickets= 2
prev_30ball_bound= 10
prev_30ball_dots= 7
prev_30ball_extras= 4
prev_30ball_runs= 60
prev_30ball_wkts= 0

score_prediction(overs,total_score,total_wickets,prev_30ball_bound,prev_30ball_dots,prev_30ball_extras,prev_30ball_runs,prev_30ball_wkts)

Average Predicted Score:  175
Maximumm Predicted Score: 193
